In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1684954695550/work


In [2]:
import pickle
import pandas as pd
import os

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

year = 2022
month = 2

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f"outputs/yellow_tripdata_{year:04d}-{month:02d}.parquet"

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
df_result = df[["ride_id"]]
df_result["predicted_duration"] = y_pred

/tmp/ipykernel_10405/1450835752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result["predicted_duration"] = y_pred


In [9]:
if (not os.path.exists("outputs")):
    os.mkdir("outputs")

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)